In [1]:
!rm -rf repo


In [ ]:
!pip install openai langchain chromadb gitpython tiktoken python-dotenv
import os
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"
os.makedirs("repo", exist_ok=True)
os.makedirs("chroma_db", exist_ok=True)
print("Setup complete ✅")
from git import Repo
repo_url = "https://github.com/pallets/flask.git"  # You can change this
Repo.clone_from(repo_url, "repo")
print("Repository cloned successfully ✅")




Setup complete ✅
Repository cloned successfully ✅


In [3]:
import os

def load_code_files(folder_path):
    code_texts = []

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith((".py", ".js", ".ts", ".html", ".css")):
                file_path = os.path.join(root, file)

                try:
                    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
                        content = f.read()

                        code_texts.append({
                            "file_name": file,
                            "file_path": file_path,
                            "content": content
                        })
                except:
                    continue

    return code_texts

code_data = load_code_files("repo")

print(f"Loaded {len(code_data)} code files ✅")


Loaded 105 code files ✅


In [4]:
code_data[0]["file_name"], code_data[0]["content"][:500]


('views.py',
 'from flask import jsonify\nfrom flask import render_template\nfrom flask import request\n\nfrom . import app\n\n\n@app.route("/", defaults={"js": "fetch"})\n@app.route("/<any(xhr, jquery, fetch):js>")\ndef index(js):\n    return render_template(f"{js}.html", js=js)\n\n\n@app.route("/add", methods=["POST"])\ndef add():\n    a = request.form.get("a", 0, type=float)\n    b = request.form.get("b", 0, type=float)\n    return jsonify(result=a + b)\n')

In [5]:
!pip install -U langchain langchain-community langchain-text-splitters
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [6]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    separators=["\nclass ", "\ndef ", "\n\n", "\n", " "]
)
documents = []

for file in code_data:
    chunks = splitter.split_text(file["content"])

    for chunk in chunks:
        documents.append({
            "content": chunk,
            "metadata": {
                "file_name": file["file_name"],
                "file_path": file["file_path"]
            }
        })

print(f"Created {len(documents)} chunks ✅")



Created 1082 chunks ✅


In [7]:
documents[0]


{'content': 'from flask import jsonify\nfrom flask import render_template\nfrom flask import request\n\nfrom . import app\n\n\n@app.route("/", defaults={"js": "fetch"})\n@app.route("/<any(xhr, jquery, fetch):js>")\ndef index(js):\n    return render_template(f"{js}.html", js=js)\n\n\n@app.route("/add", methods=["POST"])\ndef add():\n    a = request.form.get("a", 0, type=float)\n    b = request.form.get("b", 0, type=float)\n    return jsonify(result=a + b)',
 'metadata': {'file_name': 'views.py',
  'file_path': 'repo/examples/javascript/js_example/views.py'}}

In [8]:
# Install (run once)
!pip install -U sentence-transformers chromadb

# Imports
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document

# Convert your chunks into LangChain Documents
lc_docs = []

for doc in documents:
    lc_docs.append(
        Document(
            page_content=doc["content"],
            metadata=doc["metadata"]
        )
    )

print("Converted to LangChain documents ✅")

# Create Free Embedding Model
embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Create Chroma Vector Database
vector_db = Chroma.from_documents(
    documents=lc_docs,
    embedding=embedding,
    persist_directory="chroma_db"
)

print("Vector DB created using FREE embeddings ✅")

# Test Search
results = vector_db.similarity_search("template rendering", k=2)

for r in results:
    print("FILE:", r.metadata["file_name"])
    print(r.page_content[:200])
    print("------")


Converted to LangChain documents ✅


/tmp/ipython-input-2995642309.py:23: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Vector DB created using FREE embeddings ✅
FILE: templating.py
def render_template(
    template_name_or_list: str | Template | list[str | Template],
    **context: t.Any,
) -> str:
    """Render a template by name with the given context.

    :param template_nam
------
FILE: templating.py
def render_template(
    template_name_or_list: str | Template | list[str | Template],
    **context: t.Any,
) -> str:
    """Render a template by name with the given context.

    :param template_nam
------


In [9]:
# Install free LLM
!pip install -U transformers accelerate

# Import
from transformers import pipeline

qa_model = pipeline(
    "text-generation",
    model="microsoft/phi-2"
)

print("LLM loaded correctly ✅")



# Create Ask Function
def ask_question(question):

    # Retrieve relevant chunks
    results = vector_db.similarity_search(question, k=3)

    context = "\n\n".join([r.page_content for r in results])

    # Prompt
    prompt = f"""
    Answer the question based only on the following code:

    {context}

    Question: {question}
    """

    # Generate answer
    response = qa_model(prompt, max_new_tokens=200)

    return response[0]["generated_text"]

# Test
answer = ask_question("How does template rendering work?")
print(answer)


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/453 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Passing `generation_config` together with generation-related arguments=({'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=200) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


LLM loaded correctly ✅

    Answer the question based only on the following code:

    def render_template(
    template_name_or_list: str | Template | list[str | Template],
    **context: t.Any,
) -> str:
    """Render a template by name with the given context.

    :param template_name_or_list: The name of the template to render. If
        a list is given, the first name to exist will be rendered.
    :param context: The variables to make available in the template.
    """
    ctx = app_ctx._get_current_object()
    template = ctx.app.jinja_env.get_or_select_template(template_name_or_list)
    return _render(ctx, template, context)

def render_template(
    template_name_or_list: str | Template | list[str | Template],
    **context: t.Any,
) -> str:
    """Render a template by name with the given context.

    :param template_name_or_list: The name of the template to render. If
        a list is given, the first name to exist will be rendered.
    :param context: The variables to ma

In [12]:
def ask_question(question):

    results = vector_db.similarity_search(question, k=3)
    context = "\n\n".join([r.page_content for r in results])

    prompt = f"""
You are a senior software engineer.
Explain clearly how this code works.

Code:
{context}

Question: {question}

Answer:
"""

    response = qa_model(
        prompt,
        max_new_tokens=150,
        do_sample=False
    )

    # Remove the prompt from output
    full_output = response[0]["generated_text"]
    answer = full_output.split("Answer:")[-1].strip()

    return answer
print(ask_question("How does template rendering work?"))


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Passing `generation_config` together with generation-related arguments=({'max_new_tokens', 'do_sample'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=150) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Template rendering is a process of generating HTML or other markup from a template file and a set of variables. The template file contains placeholders for the variables, which are replaced with their values when the template is rendered.

In the code above, the render_template function takes a template name or a list of template names, and a set of variables to be passed to the template. It then uses the Jinja2 template engine to render the template with the given variables.

The render_template function first gets the current object of the application context, and then gets the template object using the Jinja2 environment. It then calls the _render function, which updates the template context with the given variables, and then renders the template
